# Library

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy

import torch.nn.functional as F

import torch.optim as optim
import os
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau


import warnings
warnings.filterwarnings("ignore")

# Load Data

In [20]:
transform = transforms.Compose([
        transforms.Resize(224, interpolation=2),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

batch_size = 64

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

testset=torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(dataset=testset, batch_size=batch_size,
                                         shuffle=False, num_workers=0)

dataloaders = {'train' : trainloader, 'val' : testloader }
dataset_sizes = { 'train' : len(trainset) , 'val' : len(testset) }

class_names = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')
print(class_names)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #GPU 사용가능 여부확인

Files already downloaded and verified
Files already downloaded and verified
('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


___

# Model

In [21]:
# Resnet18 모델 불러오기
model = models.resnet50(pretrained=True)

# resnet50에서 마지막에 출력되는 출력값들
num_ftrs = model.fc.in_features

# input : num_ftrs, output : 10개의 클래스로 최종 추출
model.fc = nn.Linear(num_ftrs, 10).cuda()

# model을 gpu 설정
model_ft = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)

In [22]:
# model training
def model_training(model, device, train_dataloader, optimizer, train_acc, train_losses):

    model.train()
    pbar = tqdm(train_dataloader)
    correct = 0
    processed = 0
    running_loss = 0.0

    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        y_pred = model(data)
        loss = F.cross_entropy(y_pred, target)


        train_losses.append(loss)
        loss.backward()
        optimizer.step()
        correct += y_pred.argmax(dim=1).eq(target).sum()
        processed += len(data)
        # print statistics
        running_loss += loss.item()
        pbar.set_description(desc=f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
        train_acc.append(100*correct/processed)

def model_testing(model, device, test_dataloader, test_acc, test_losses, misclassified = []):

    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():

        for index, (data, target) in enumerate(test_dataloader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            test_loss += F.cross_entropy(output, target)
            correct += pred.eq(target.view_as(pred)).sum()
    test_loss /= len(test_dataloader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_dataloader.dataset),
        100. * correct / len(test_dataloader.dataset)))

    test_acc.append(100. * correct / len(test_dataloader.dataset))

# Train & Test the Data

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=True)
# scheduler = StepLR(optimizer, step_size=15, gamma=0.1)

train_acc = []
train_losses = []
test_acc = []
test_losses = []

EPOCHS = 10

for i in range(EPOCHS):
    print(f'EPOCHS : {i}')
    model_training(model, device, trainloader, optimizer, train_acc, train_losses)
    scheduler.step(train_losses[-1])
    model_testing(model, device, testloader, test_acc, test_losses)

EPOCHS : 0


Loss=0.08851327747106552 Batch_id=781 Accuracy=87.57: 100%|██████████| 782/782 [03:07<00:00,  4.18it/s]



Test set: Average loss: 0.0026, Accuracy: 9437/10000 (94.37%)

EPOCHS : 1


Loss=0.023596376180648804 Batch_id=781 Accuracy=96.54: 100%|██████████| 782/782 [03:08<00:00,  4.15it/s]



Test set: Average loss: 0.0020, Accuracy: 9556/10000 (95.56%)

EPOCHS : 2


...

Epoch 00008: reducing learning rate of group 0 to 5.0000e-05.

Test set: Average loss: 0.0024, Accuracy: 9594/10000 (95.94%)

EPOCHS : 8


Loss=0.0002972630027215928 Batch_id=781 Accuracy=99.95: 100%|██████████| 782/782 [03:19<00:00,  3.92it/s] 



Test set: Average loss: 0.0022, Accuracy: 9609/10000 (96.09%)

EPOCHS : 9


Loss=0.04367726668715477 Batch_id=781 Accuracy=99.96: 100%|██████████| 782/782 [03:21<00:00,  3.87it/s]   



Test set: Average loss: 0.0022, Accuracy: 9608/10000 (96.08%)



# Save Model

In [24]:
# save model
torch.save(model, "./model.pt")

# save model state_dict
torch.save(model.state_dict(), "./model_state_dict.pt")

# Load Model

In [ ]:
# load model
model = torch.load("./model.pt")

# load model state_dict
model2 = MyModel()
model2.load_state_dict(torch.load("./model_state_dict.pt"))